In [11]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

# Please Modify the following accordingly 
# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = False
delete_problematic = False

# Which status to change
change_status = 'released to project'

sets_in_scope =  ['4DNESYTWHUH6', '4DNES21D8SP8']
search_url = ""  # "/search/type=AAAAAAAA"


if sets_in_scope:
    set_to_release = [ff_utils.get_metadata(i, my_auth)['uuid'] for i in sets_in_scope]
elif search_url:
    set_to_release = [i['uuid'] for i in ff_utils.search_metadata(search_url, my_auth)]

store={}
item_uuids=[]
store, uuids = ff_utils.expand_es_metadata(set_to_release, my_auth, store_frame='object',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])

print(len(store['experiment_set_replicate']), 'exp sets for status change')
print(len(uuids), 'items collected')
time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

2 exp sets for status change
711 items collected
5.9 sec for collection


In [16]:
# TODO
# Check audits

# to decide which items to ignore as they have a 'higher' status
STATUS_LEVEL = {
    # standard_status
    "released": 10, "current": 10,
    "released to project": 9,
    "pre-release": 8,
    "planned": 6, "submission in progress": 6,
    "in review by lab": 4,
    "revoked": 0, "archived": 0,"deleted": 0, "obsolete": 0, "replaced": 0, "archived to project": 0,
    # additional file statuses
    'to be uploaded by workflow': 4, 'uploading': 4, 'uploaded': 4, 'upload failed': 4, 'draft': 4}

change_level = STATUS_LEVEL.get(change_status, 1)

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = [i for i in store if i.startswith('experiment') and not i.startswith('experiment_set')]
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        try:
            # to do skip sbg runs
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)
        except:
            print('sbg run skipping')

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['file_format'] == '/file-formats/pairs/':
            if not a_file.get('quality_metric'):
                print(a_file['accession'], 'missing Pairsqc')
        if not a_file.get('source_experiments'):
            print(a_file['accession'], 'missing source experiments')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
print('  Following run types are found:')
for wf in set([i['display_title'].split(' ')[0] for i in store.get('workflow_run_awsem')]):
           print('    ' + wf)
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i, x['uuid'], x['status']] for x in store[i] if STATUS_LEVEL.get(x['status']) == 0]
    if weird:
        for case in weird:
            print(case)
        print()

EXPSET CHECK

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK

WFR CHECK
  Following run types are found:
    pairsqc-single
    bwa-mem
    hi-c-processing-bam
    hi-c-processing-pairs-nore
    fastqc-0-11-4-1
    md5

REPORT NUMBERS AND CHECK STATUS
lab 7
organism 1
ontology 2
biosource 2
quality_metric_fastqc 92
vendor 3
ontology_term 15
experiment_hi_c 20
biosample_cell_culture 4
workflow 9
individual_human 2
file_reference 2
image 4
software 10
static_section 3
biosample 4
enzyme 1
protocol 10
file_processed 94
file_fastq 92
user 15
award 6
file_format 14
quality_metric_pairsqc 23
experiment_set_replicate 2
workflow_run_awsem 274


In [19]:
for i in store:
    for x in store[i]:
        search = str(x)
        if 'ruan' in search.lower():
            print(i, x['uuid'], x.get('display_title'))

lab abd48785-b0e5-4453-be14-30d37a516bf3 Yijun Ruan, JAX
user 68b09572-0c3e-48c7-90a9-7bad21d6bac1 Rafael Casellas
user d8ac229e-ec08-4411-be38-dc779520ea62 Yijun Ruan
award 029a2578-43dc-4343-8f41-694518cce304 NUCLEOME POSITIONING SYSTEM FOR SPATIOTEMPORAL GENOME ORGANIZATION AND REGULATION


In [20]:
# Check status
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))
    
       

lab                       Out of 7, 7 skipped, 0 matching, 0 unusual, and 0 needs change
organism                  Out of 1, 1 skipped, 0 matching, 0 unusual, and 0 needs change
ontology                  Out of 2, 2 skipped, 0 matching, 0 unusual, and 0 needs change
biosource                 Out of 2, 2 skipped, 0 matching, 0 unusual, and 0 needs change
quality_metric_fastqc     Out of 92, 0 skipped, 92 matching, 0 unusual, and 0 needs change
vendor                    Out of 3, 3 skipped, 0 matching, 0 unusual, and 0 needs change
ontology_term             Out of 15, 15 skipped, 0 matching, 0 unusual, and 0 needs change
experiment_hi_c           Out of 20, 0 skipped, 20 matching, 0 unusual, and 0 needs change
biosample_cell_culture    Out of 4, 0 skipped, 4 matching, 0 unusual, and 0 needs change
workflow                  Out of 9, 9 skipped, 0 matching, 0 unusual, and 0 needs change
individual_human          Out of 2, 2 skipped, 0 matching, 0 unusual, and 0 needs change
file_reference 

In [22]:
# If you want to patch the status, change action to True
action = True

reviewed = ""
reviewed = input('Status will change to {}\nDid another wrangler review this release? (y/n):'.format(change_status))
if reviewed != 'y':
    raise KeyError('A key step is missing!')
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released' and a_type in ['publication']:
                    patch_data = {'status': 'current'}
                   
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth) 

        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))

Status will change to released to project
Did another wrangler review this release? (y/n):y
lab                       Out of 7, 7 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
organism                  Out of 1, 1 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
ontology                  Out of 2, 2 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
biosource                 Out of 2, 2 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
quality_metric_fastqc     Out of 92, 0 skipped, 92 matching, 0 unusual, and 0 UPDATED with status
vendor                    Out of 3, 3 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
ontology_term             Out of 15, 15 skipped, 0 matching, 0 unusual, and 0 UPDATED with status
experiment_hi_c           Out of 20, 0 skipped, 20 matching, 0 unusual, and 0 UPDATED with status
biosample_cell_culture    Out of 4, 0 skipped, 4 matching, 0 unusual, and 0 UPDATED with status
workflow                  Out of 9, 9 